# 뉴스 크롤링

In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [ ]:
!apt-get update
!apt install chromium-chromedriver

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)

In [64]:
# 기사 링크 크롤링
# 카테고리당 100개 = 5페이지

url_list = []   # url 저장 리스트
a_list = []     # a 태그 저장 리스트
category_list = []
categoty_name = ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학"]

for category in range(3, 5):     # 6
    for page in range(1, 2):  # 1, 6
        url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={100 + category}#&date=%2000:00:00&page={page}'
        browser.get(url)

        time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")
        a_list.extend(soup.select(".type06_headline dt+dt a"))
        a_list.extend(soup.select(".type06 dt+dt a"))

        category_list.extend([categoty_name[category] for _ in range(20)])
        print(f"카테고리 : {categoty_name[category]} {page}페이지")

for a in a_list:  # 링크
    url_list.append(a["href"])

카테고리 : 생활/문화 1페이지
카테고리 : 세계 1페이지


In [25]:
print(url_list)

['https://n.news.naver.com/mnews/article/001/0014300200?sid=103', 'https://n.news.naver.com/mnews/article/009/0005207597?sid=103', 'https://n.news.naver.com/mnews/article/215/0001131907?sid=103', 'https://n.news.naver.com/mnews/article/296/0000070998?sid=103', 'https://n.news.naver.com/mnews/article/584/0000024777?sid=103', 'https://n.news.naver.com/mnews/article/005/0001648481?sid=103', 'https://n.news.naver.com/mnews/article/009/0005207572?sid=103', 'https://n.news.naver.com/mnews/article/009/0005207566?sid=103', 'https://n.news.naver.com/mnews/article/009/0005207565?sid=103', 'https://n.news.naver.com/mnews/article/009/0005207564?sid=103', 'https://n.news.naver.com/mnews/article/009/0005207563?sid=103', 'https://n.news.naver.com/mnews/article/009/0005207561?sid=103', 'https://n.news.naver.com/mnews/article/009/0005207557?sid=103', 'https://n.news.naver.com/mnews/article/030/0003151426?sid=103', 'https://n.news.naver.com/mnews/article/296/0000070996?sid=103', 'https://n.news.naver.co

In [65]:
# 수집한 링크의 제목, 본문, 날짜 크롤링

title = []
content = []
date = []

for url in url_list:
      browser.get(url)

      time.sleep(0.5)

      soup = BeautifulSoup(browser.page_source, "html.parser")

      title.extend(soup.select("#title_area span"))       # 제목

      c = soup.find_all(attrs={"id" : "dic_area"})        # 본문

      while c[0].find(attrs={"class" : "end_photo_org"}):         # 이미지 있는 만큼
        c[0].find(attrs={"class" : "end_photo_org"}).decompose()  # 본문 이미지에 있는 글자 없애기

      while c[0].find(attrs={"class" : "vod_player_wrap"}):       # 영상 있는 만큼
        c[0].find(attrs={"class" : "vod_player_wrap"}).decompose()  # 본문 영상에 있는 글자 없애기

      content.extend(c)

      date.extend(soup.select("._ARTICLE_DATE_TIME"))     # 날짜

In [66]:
# 위에서 수집한 태그 속에서 내용만 뽑아서 담는다.

def clean(article):   # 전처리
  article = re.sub('\n','',article)
  article = re.sub('\t','',article)
  article = re.sub('\u200b','',article)
  article = re.sub('\xa0','',article)
  article = re.sub('([a-zA-Z])','',article)
  article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)
  article = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',article)

  return article

title_list = []     # 제목
content_list = []   # 본문
date_list = []      # 날짜

for t in title:
  title_list.append(t.text)

for c in content:
  content_list.append(clean(c.text))

for d in date:
  date_list.append(d["data-date-time"])

In [ ]:
# 데이터 프레임으로 변환

article_df = pd.DataFrame({"카테고리" : category_list,
                           "날짜" : date_list,
                           "제목" : title_list,
                           "본문" : content_list,
                           "링크" : url_list})

article_df

# 전처리

**KoNLPy**
- 한국어 정보처리를 위한 파이썬 패키지
- https://konlpy.org/ko/latest/api/konlpy.tag/#okt-class

In [10]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 45.6 MB/s eta 0:00:00


In [83]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# CountVectorizer : 텍스트 빈도만을 카운트해서 벡터화
# TfidfVectorizer : 자주 나오는 단어에 높은 가중치, 모든 문서에서 자주 나오는 단어에 패널티

In [88]:
okt = Okt()
nouns_list = []   # 명사 리스트

for content in article_df["본문"]:

  content = clean(content)         # 전처리
  nouns_list.append(okt.nouns(content))     # 명사 추출, 리스트 반환

article_df["명사"] = nouns_list   # 데이터 프레임에 추가

[['앵커', '주류', '소비', '연말', '대표', '서민', '술', '소주', '맥주', '가격', '소문', '무성', '주류', '값', '도', '미노', '인상', '현실', '식당', '주인', '사', '손님', '고민', '전서', '기자', '보도', '기자', '눈', '오른', '술', '가격', '직장인', '회식', '입', '안태홍', '서울시', '공릉동', '옛날', '때', '맘', '요즘', '병', '정도', '원', '업계', '위', '맥주', '위', '하이트진로', '다음', '달', '소주', '맥주', '출고', '하이트진로', '관계자', '폭', '원가', '상승', '요인', '발생', '정부', '물가', '안정', '노력', '발', '소비자', '부담', '최소', '선', '선두', '업체', '가격', '롯데', '주류', '인상', '간문제', '소주', '맥주', '출고', '정도', '식당', '술집', '판매', '이', '크게', '오른', '각각', '원', '원', '선', '것', '예상', '서울시', '식당', '술', '가격', '정도', '매출', '별', '도움', '안', '수', '가격', '오비', '맥주', '식당', '판매', '병', '선', '오른', '상태', '어제', '화장품', '오늘', '주류', '음료', '세제', '등', '사방', '인상', '소식', '가운데', '원가', '부담', '소비', '부진', '속', '실적', '악화', '온', '기업', '정부', '더', '이상', '통제', '것', '분석', '전서', '당신', '제보', '뉴스', '여러분', '제보', '홈페이지'], ['중부', '지역', '미세먼지', '수요일', '새벽', '밤', '사이', '수도권', '강원', '영서', '중심', '가끔', '비', '곳', '돌풍', '천둥', '번개', '곳도', '것', '전망', '또', '서울', '경기도', '등

In [92]:
text = [" ".join(noun) for noun in article_df["명사"]]  # 명사 열을 하나의 리스트에 담는다.

tfidf_vectorizer = TfidfVectorizer(min_df = 5, ngram_range=(1, 5))

['앵커 주류 소비 연말 대표 서민 술 소주 맥주 가격 소문 무성 주류 값 도 미노 인상 현실 식당 주인 사 손님 고민 전서 기자 보도 기자 눈 오른 술 가격 직장인 회식 입 안태홍 서울시 공릉동 옛날 때 맘 요즘 병 정도 원 업계 위 맥주 위 하이트진로 다음 달 소주 맥주 출고 하이트진로 관계자 폭 원가 상승 요인 발생 정부 물가 안정 노력 발 소비자 부담 최소 선 선두 업체 가격 롯데 주류 인상 간문제 소주 맥주 출고 정도 식당 술집 판매 이 크게 오른 각각 원 원 선 것 예상 서울시 식당 술 가격 정도 매출 별 도움 안 수 가격 오비 맥주 식당 판매 병 선 오른 상태 어제 화장품 오늘 주류 음료 세제 등 사방 인상 소식 가운데 원가 부담 소비 부진 속 실적 악화 온 기업 정부 더 이상 통제 것 분석 전서 당신 제보 뉴스 여러분 제보 홈페이지',
 '중부 지역 미세먼지 수요일 새벽 밤 사이 수도권 강원 영서 중심 가끔 비 곳 돌풍 천둥 번개 곳도 것 전망 또 서울 경기도 등 대부분 중부 지역 미세먼지 농도 것 보되 기상청 내일 새벽 북부 강원 북부 비 시작 아침 그 밖 수도권 강원 영서로 비 확대 저녁 대부분 예보 예상 강수량 경기 북부 강원 영서 북부 서울 인천 경기 남부 강원 남부 충북 북부 내외 다비 지역 돌풍 천둥 번개 곳 대기 불안 정이 일부 경기 북 동부 강원 영서 북부 우박 곳도 전망 당분간 기온 평년 최저 기온 도 최고 기온 도보 조금 남부 지방 중심 낮 밤 기온 도 내외 내일 중부지방 대체로 남부 지방 제주도 대체로 맑은 가운데 아침 최저 기온 도 낮 최고 기온 것 지역 아침 최저 기온 서울 도 인천 도 수원 도 춘천 도 강릉 도 청주 도 대전 도 전주 도 광주 도 대구 도 부산 도 제주 도 낮 최고 기온 서울 도 인천 도 수원 도 춘천 도 강릉 도 청주 도 대전 도 전주 도 광주 도 대구 도 부산 도 제주 도 새벽 오전 사이 충남 륙 전라 경남 륙 가시거리 미만 안개 그 밖 지역 가시거리 미만 안개 곳 전망 미세먼지 잔류 미세먼지 대기 정

# 군집화

In [86]:
from sklearn.cluster import DBSCAN
# DBSCAN : 밀도 기반 클러스터링

In [87]:
vector

TfidfVectorizer(min_df=2)